# ant

> Meta Tools for AI agents

In [ ]:
#| default_exp core

In [ ]:
#| export
import requests
from cosette import contents, Chat


### Getting L402 URI Info

In [ ]:
#| export
def get_l402_uri_info(uri: str) -> dict:
    """
   Returns the information of the L402 URI resource.

    Args:
    uri (str): The L402 URI to describe.

    Returns:
    dict: A dictionary containing the description of the L402 URI.
    """
    if not uri.startswith("l402://"):
        raise ValueError("Invalid L402 URI format")
    
    scheme = "http://" if "localhost" in uri else "https://"
    http_url = uri.replace("l402://", scheme, 1)
    
    response = requests.get(http_url)
    response.raise_for_status()
    
    return response.json()

    

In [ ]:
uri = "l402://api.fewsats.com/v0/gateway/f12e5deb-b07b-4af4-a4f2-3fbf076228a9/info" # web scraping endpoint

info = get_l402_uri_info(uri)
info['access'].pop('authentication', None)
info

{'access': {'endpoint': 'https://api.fewsats.com/v0/gateway/access/f12e5deb-b07b-4af4-a4f2-3fbf076228a9',
  'method': 'POST'},
 'content_type': 'api',
 'cover_url': '',
 'description': 'Scrape a given URL. Pass the URL as JSON in the request body as follows:\n```\n {"url": url }\n```',
 'name': 'Web Scraper TF',
 'pricing': [{'amount': 1, 'currency': 'USD'}],
 'version': '0.1'}

### Generating Python Function

In [ ]:
#| export

import json 

func_generation_sp = """You are an AI assistant specialized in creating Python functions based on L402 info inputs. 
When given an L402 info dictionary, your task is to generate a Python function that can access the specified endpoint. 
Follow these guidelines:

1. Create a function name that relates to the resource or action described in the 'name' field making it specific enough to avoid conflicts.
2. Use the 'endpoint' field to determine the URL for the request.
3. Use the 'method' field to determine the HTTP method for the request.
4. Handle any required parameters for the endpoint request by passing them as function params.
5. Write a docstring that includes:
   - A brief description of the function's purpose
   - Parameters with their types and descriptions
   - Return value with its type and description
6. Do not import the requests library. Assume it is available in the global scope.
7. Handle potential errors and exceptions appropriately.
8. Do not handle any authentication or authorization in the function.
9. Return the response from the endpoint.
10. Use L402 request client to send the request and configure exactly like this:

L402 Request configuration:
```
l402_requests.configure(
   preimage_provider=AlbyAPI(api_key=os.getenv("ALBY_TOKEN")),
   credentials_service=SqliteCredentialsService(":memory:")
)
```

Example input:
```
{'access': {'authentication': {'format': 'L402 {credentials}:{proof_of_payment}',
   'header': 'Authorization',
   'protocol': 'L402'},
  'endpoint': 'https://blockbuster.fewsats.com/video/stream/79c816f77fdc4e66b8cd18ad67537936',
  'method': 'POST'},
 'content_type': 'video',
 'cover_url': 'https://pub-3c55410f5c574362bbaa52948499969e.r2.dev/cover-images/79c816f77fdc4e66b8cd18ad67537936',
 'description': 'Lex Fridman Podcast full episode:    • Andrew Huberman: Focus, Controversy, ',
 'name': 'How to focus and think deeply | Andrew Huberman and Lex Fridman',
 'pricing': [{'amount': 1, 'currency': 'USD'}],
 'version': '1.0'}
```

Example output:
```
from l402.client import requests as l402_requests
from l402.client.preimage_provider import AlbyAPI
from l402.client.credentials import SqliteCredentialsService
import requests
import os

l402_requests.configure(
   preimage_provider=AlbyAPI(api_key=os.getenv("ALBY_TOKEN")),
   credentials_service=SqliteCredentialsService()
)
 def stream_video_huberman() -> dict:
    \"\"\"
    Stream the video of the podcast episode featuring Andrew Huberman and Lex Fridman.

    This function sends a POST request to the specified endpoint.
    Returns:
    - requests.Response: The response object containing the server's response to the request.
    \"\"\"
    endpoint = "https://blockbuster.fewsats.com/video/stream/79c816f77fdc4e66b8cd18ad67537936"
    try:
        response = l402_requests.post(endpoint, headers=headers)
        response.raise_for_status()  # Raise an error for bad responses
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None
```
Your response should be a complete Python function definition, including imports, type hints, docstring, and function body. Only output the function definition, no other text."""


In [ ]:
#| export

def generate_python_function(info: dict) -> str:
   """
   Generates a Python function based on the provided L402 info dictionary.

   Args:
   info (dict): The L402 info dictionary containing the details of the resource.

   Returns:
   str: A Python function definition as a string.
   """
   # we remove authentication because it's handled by the L402 client
   # and it only confuses the LLM
   info['access'].pop('authentication', None)

   chat = Chat('gpt-4o-mini', sp=func_generation_sp)
   return chat(json.dumps(info))


In [ ]:

func_code = generate_python_function(info)

### Extracting the function code

In [ ]:
response = contents(func_code)
response

'```python\nfrom l402.client import requests as l402_requests\nfrom l402.client.preimage_provider import AlbyAPI\nfrom l402.client.credentials import SqliteCredentialsService\nimport requests\nimport os\n\nl402_requests.configure(\n   preimage_provider=AlbyAPI(api_key=os.getenv("ALBY_TOKEN")),\n   credentials_service=SqliteCredentialsService()\n)\ndef scrape_web_url(url: str) -> dict:\n    """\n    Scrape a given URL by sending a POST request to the specified endpoint.\n\n    This function sends a POST request with the URL in the request body as JSON.\n    \n    Parameters:\n    - url (str): The URL to be scraped.\n\n    Returns:\n    - dict: The JSON response from the server containing the scraped data.\n    """\n    endpoint = "https://api.fewsats.com/v0/gateway/access/f12e5deb-b07b-4af4-a4f2-3fbf076228a9"\n    json_body = {"url": url}\n    try:\n        response = l402_requests.post(endpoint, json=json_body)\n        response.raise_for_status()  # Raise an error for bad responses\n 

In [ ]:
#| export

import re
def extract_function_code(func_code):
    code_pattern = re.compile(r'```python\n(.*?)```', re.DOTALL)
    match = code_pattern.search(func_code)
    return match.group(1).strip() if match else func_code


In [ ]:
extracted_code = extract_function_code(response)
print(extracted_code)

from l402.client import requests as l402_requests
from l402.client.preimage_provider import AlbyAPI
from l402.client.credentials import SqliteCredentialsService
import requests
import os

l402_requests.configure(
   preimage_provider=AlbyAPI(api_key=os.getenv("ALBY_TOKEN")),
   credentials_service=SqliteCredentialsService()
)

def web_scraper_tf(url: str) -> dict:
    """
    Scrape a given URL using the Web Scraper TF service.

    This function sends a POST request to the specified endpoint with the URL to scrape.
    
    Parameters:
    - url (str): The URL to be scraped.

    Returns:
    - dict: The JSON response containing the result of the scraping operation.
    """
    endpoint = "https://api.fewsats.com/v0/gateway/access/f12e5deb-b07b-4af4-a4f2-3fbf076228a9"
    headers = {'Content-Type': 'application/json'}
    payload = {"url": url}
    
    try:
        response = l402_requests.post(endpoint, json=payload, headers=headers)
        response.raise_for_status()  # Raise an e

### Creating the function

In [ ]:
#| export

import importlib.util
import re
import os

def create_func(code_string):
    # Ensure the funcs directory exists
    os.makedirs('.funcs', exist_ok=True)

    # Extract Python code from Markdown code blocks if present
    code_pattern = re.compile(r'```python\n(.*?)```', re.DOTALL)
    match = code_pattern.search(code_string)
    extracted_code = match.group(1).strip() if match else code_string

    # Create a temporary file name

    function_name = extracted_code.split('def ')[1].split('(')[0].strip()

    temp_file = f'.funcs/{function_name}.py'

    # Write the code to a temporary file
    with open(temp_file, 'w') as f:
        f.write(extracted_code)

    # Import the function
    spec = importlib.util.spec_from_file_location("temp_module", temp_file)
    temp_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(temp_module)

    # Get the function
    function_name = extracted_code.split('def ')[1].split('(')[0].strip()
    return getattr(temp_module, function_name)



In [ ]:

gf = create_func(extracted_code)


In [ ]:
r = gf('https://www.fewsats.com/')
r

{'success': True,
 'data': {'markdown': "![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/66475f66cdbcb00d7bf2fd44_Employee%20data.png)\n\n[Learn More](#)![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/6645a4a7082c15bec7e9fdda_arrow-right-icon.png)\n\n\n![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/6645a4a7082c15bec7e9fdda_arrow-right-icon.png)\n\n[Learn More](#)![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/6645a4a7082c15bec7e9fdda_arrow-right-icon.png)\n\n\n![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/6645a4a7082c15bec7e9fdda_arrow-right-icon.png)\n\n![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/664773c569f5d9dbac6655a0_Employee%20data-2.png)\n\n![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/664762e683491e5b21c08f31_Employee%20data-3.png)\n\n[Learn More](#)![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/6645a4a7082c15bec7e9fdda_arrow-right-icon.png)\

## Adding Tools to the Agent

In [ ]:
tools = [get_l402_uri_info]
chat = Chat('gpt-4o-mini', sp='You are a helpful assistant that provides information about L402 URIs and your available tools.', tools=tools)

chat("which tools do you have available?")

I have access to a tool that retrieves information about L402 URIs. Specifically, I can provide details about a given L402 URI by describing its properties. If you have a specific L402 URI in mind, I can help you get information about it.

<details>

- id: chatcmpl-AIc5Z7maj5Kam0f8T1fxETNCeiJf6
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I have access to a tool that retrieves information about L402 URIs. Specifically, I can provide details about a given L402 URI by describing its properties. If you have a specific L402 URI in mind, I can help you get information about it.', refusal=None, role='assistant', function_call=None, tool_calls=None))]
- created: 1728999681
- model: gpt-4o-mini-2024-07-18
- object: chat.completion
- service_tier: None
- system_fingerprint: fp_e2bde53e6e
- usage: CompletionUsage(completion_tokens=53, prompt_tokens=111, total_tokens=164, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0))

</details>

In [ ]:
chat(f'What is the info of: {uri}')


Here is the information for the L402 URI `l402://api.fewsats.com/v0/gateway/f12e5deb-b07b-4af4-a4f2-3fbf076228a9/info`:

- **Name**: Web Scraper TF
- **Description**: Scrape a given URL. Pass the URL as JSON in the request body as follows:
  ```json
  {"url": url }
  ```
- **Content Type**: api
- **Version**: 0.1
- **Access**:
  - **Authentication**:
    - **Format**: L402 {credentials}:{proof_of_payment}
    - **Header**: Authorization
    - **Protocol**: L402
  - **Endpoint**: [https://api.fewsats.com/v0/gateway/access/f12e5deb-b07b-4af4-a4f2-3fbf076228a9](https://api.fewsats.com/v0/gateway/access/f12e5deb-b07b-4af4-a4f2-3fbf076228a9)
  - **Method**: POST
- **Pricing**: 
  - **Amount**: 1
  - **Currency**: USD
- **Cover URL**: (not provided)

If you need more information or have another URI to inquire about, feel free to ask!

<details>

- id: chatcmpl-AIc61tskvYg7i2f0fNFguyo72xTKC
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Here is the information for the L402 URI `l402://api.fewsats.com/v0/gateway/f12e5deb-b07b-4af4-a4f2-3fbf076228a9/info`:\n\n- **Name**: Web Scraper TF\n- **Description**: Scrape a given URL. Pass the URL as JSON in the request body as follows:\n  ```json\n  {"url": url }\n  ```\n- **Content Type**: api\n- **Version**: 0.1\n- **Access**:\n  - **Authentication**:\n    - **Format**: L402 {credentials}:{proof_of_payment}\n    - **Header**: Authorization\n    - **Protocol**: L402\n  - **Endpoint**: [https://api.fewsats.com/v0/gateway/access/f12e5deb-b07b-4af4-a4f2-3fbf076228a9](https://api.fewsats.com/v0/gateway/access/f12e5deb-b07b-4af4-a4f2-3fbf076228a9)\n  - **Method**: POST\n- **Pricing**: \n  - **Amount**: 1\n  - **Currency**: USD\n- **Cover URL**: (not provided)\n\nIf you need more information or have another URI to inquire about, feel free to ask!', refusal=None, role='assistant', function_call=None, tool_calls=None))]
- created: 1728999709
- model: gpt-4o-mini-2024-07-18
- object: chat.completion
- service_tier: None
- system_fingerprint: fp_e2bde53e6e
- usage: CompletionUsage(completion_tokens=296, prompt_tokens=483, total_tokens=779, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0))

</details>

In [ ]:
# Step 1: Get L402 URI info
info = get_l402_uri_info(uri)
print("L402 URI info retrieved successfully.")
info


L402 URI info retrieved successfully.


{'access': {'authentication': {'format': 'L402 {credentials}:{proof_of_payment}',
   'header': 'Authorization',
   'protocol': 'L402'},
  'endpoint': 'https://api.fewsats.com/v0/gateway/access/f12e5deb-b07b-4af4-a4f2-3fbf076228a9',
  'method': 'POST'},
 'content_type': 'api',
 'cover_url': '',
 'description': 'Scrape a given URL. Pass the URL as JSON in the request body as follows:\n```\n {"url": url }\n```',
 'name': 'Web Scraper TF',
 'pricing': [{'amount': 1, 'currency': 'USD'}],
 'version': '0.1'}

In [ ]:
# Step 2: Generate Python function
func_code = generate_python_function(info)
print("Python function generated successfully.")
func_code

Python function generated successfully.


```python
from l402.client import requests as l402_requests
from l402.client.preimage_provider import AlbyAPI
from l402.client.credentials import SqliteCredentialsService
import requests
import os

l402_requests.configure(
   preimage_provider=AlbyAPI(api_key=os.getenv("ALBY_TOKEN")),
   credentials_service=SqliteCredentialsService()
)
def scrape_web_url(url: str) -> dict:
    """
    Scrape a given URL by sending a POST request to the specified endpoint.

    This function sends a POST request with the URL in the request body as JSON.
    
    Parameters:
    - url (str): The URL to be scraped.

    Returns:
    - dict: The JSON response from the server containing the scraped data.
    """
    endpoint = "https://api.fewsats.com/v0/gateway/access/f12e5deb-b07b-4af4-a4f2-3fbf076228a9"
    json_body = {"url": url}
    try:
        response = l402_requests.post(endpoint, json=json_body)
        response.raise_for_status()  # Raise an error for bad responses
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None
```

<details>

- id: chatcmpl-AIc69oXLPq7iRjS0ZmsxF016MRJmk
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```python\nfrom l402.client import requests as l402_requests\nfrom l402.client.preimage_provider import AlbyAPI\nfrom l402.client.credentials import SqliteCredentialsService\nimport requests\nimport os\n\nl402_requests.configure(\n   preimage_provider=AlbyAPI(api_key=os.getenv("ALBY_TOKEN")),\n   credentials_service=SqliteCredentialsService()\n)\ndef scrape_web_url(url: str) -> dict:\n    """\n    Scrape a given URL by sending a POST request to the specified endpoint.\n\n    This function sends a POST request with the URL in the request body as JSON.\n    \n    Parameters:\n    - url (str): The URL to be scraped.\n\n    Returns:\n    - dict: The JSON response from the server containing the scraped data.\n    """\n    endpoint = "https://api.fewsats.com/v0/gateway/access/f12e5deb-b07b-4af4-a4f2-3fbf076228a9"\n    json_body = {"url": url}\n    try:\n        response = l402_requests.post(endpoint, json=json_body)\n        response.raise_for_status()  # Raise an error for bad responses\n        return response.json()\n    except requests.exceptions.RequestException as e:\n        print(f"An error occurred: {e}")\n        return None\n```', refusal=None, role='assistant', function_call=None, tool_calls=None))]
- created: 1728999717
- model: gpt-4o-mini-2024-07-18
- object: chat.completion
- service_tier: None
- system_fingerprint: fp_e2bde53e6e
- usage: CompletionUsage(completion_tokens=268, prompt_tokens=892, total_tokens=1160, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0))

</details>

In [ ]:

# Step 3: Extract function code
extracted_code = extract_function_code(get_text(func_code))
print("Function code extracted successfully.")
extracted_code

Function code extracted successfully.


'from l402.client import requests as l402_requests\nfrom l402.client.preimage_provider import AlbyAPI\nfrom l402.client.credentials import SqliteCredentialsService\nimport requests\nimport os\n\nl402_requests.configure(\n   preimage_provider=AlbyAPI(api_key=os.getenv("ALBY_TOKEN")),\n   credentials_service=SqliteCredentialsService()\n)\ndef scrape_web_url(url: str) -> dict:\n    """\n    Scrape a given URL by sending a POST request to the specified endpoint.\n\n    This function sends a POST request with the URL in the request body as JSON.\n    \n    Parameters:\n    - url (str): The URL to be scraped.\n\n    Returns:\n    - dict: The JSON response from the server containing the scraped data.\n    """\n    endpoint = "https://api.fewsats.com/v0/gateway/access/f12e5deb-b07b-4af4-a4f2-3fbf076228a9"\n    json_body = {"url": url}\n    try:\n        response = l402_requests.post(endpoint, json=json_body)\n        response.raise_for_status()  # Raise an error for bad responses\n        retu

In [ ]:

# Step 4: Create the function
cf = create_func(extracted_code)
print("Function created successfully.")

Function created successfully.


In [ ]:
tools = [get_l402_uri_info, cf]

In [ ]:
chat = Chat('gpt-4o-mini', sp='You are a helpful assistant that provides information about L402 URIs and your available tools.', tools=tools)


In [ ]:
chat("which tools do you have available?")

I have access to the following tools:

1. **get_l402_uri_info**: This tool retrieves information about an L402 URI resource. You provide it with an L402 URI, and it returns a description of that URI.

2. **scrape_web_url**: This tool scrapes a given URL by sending a POST request to a specified endpoint with the URL in the request body as JSON. It returns the JSON response containing the scraped data. 

If you have specific queries or tasks in mind, feel free to ask!

<details>

- id: chatcmpl-AIc6Djj1JgjClghDH50G4cn6QfUo5
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I have access to the following tools:\n\n1. **get_l402_uri_info**: This tool retrieves information about an L402 URI resource. You provide it with an L402 URI, and it returns a description of that URI.\n\n2. **scrape_web_url**: This tool scrapes a given URL by sending a POST request to a specified endpoint with the URL in the request body as JSON. It returns the JSON response containing the scraped data. \n\nIf you have specific queries or tasks in mind, feel free to ask!', refusal=None, role='assistant', function_call=None, tool_calls=None))]
- created: 1728999721
- model: gpt-4o-mini-2024-07-18
- object: chat.completion
- service_tier: None
- system_fingerprint: fp_e2bde53e6e
- usage: CompletionUsage(completion_tokens=109, prompt_tokens=194, total_tokens=303, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0))

</details>

In [ ]:
chat.toolloop("can you scrape this url for me: https://www.fewsats.com/")

I successfully scraped the URL https://www.fewsats.com/. Here is the scraped data:

### Title
**Monetize your Files, Databases**

### Content (Markdown)
```markdown
![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/66475f66cdbcb00d7bf2fd44_Employee%20data.png)

[Learn More](#)![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/6645a4a7082c15bec7e9fdda_arrow-right-icon.png)

![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/6645a4a7082c15bec7e9fdda_arrow-right-icon.png)
[Learn More](#)![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/6645a4a7082c15bec7e9fdda_arrow-right-icon.png)

![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/6645a4a7082c15bec7e9fdda_arrow-right-icon.png)
![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/664773c569f5d9dbac6655a0_Employee%20data-2.png)

![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/664762e683491e5b21c08f31_Employee%20data-3.png)
[Learn More](#)![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/6645a4a7082c15bec7e9fdda_arrow-right-icon.png)

The L402 protocol builds on top of HTTP and the Lightning network to enable internet-first, machine friendly paywalls.

Internet-Native Paywalls

The L402 protocol builds on top of HTTP and the Lightning network to enable internet-first, machine friendly paywalls.

L402 is an open protocol that anyone can build upon, fostering a flourishing ecosystem of innovative solutions.

AI systems are now key consumers of software, driving demand for more intelligent, contextual data and tools. With one integration, Fewsats unlocks a universe of resources to supercharge these AI consumers.

**Revolutionize your bots with Retrieval Augmented Generation (RAG) and in-context information.**
```

### Metadata
- **Source URL**: [Fewsats](https://www.fewsats.com/)
- **Status Code**: 200

If you need more specific information or further assistance, let me know!

<details>

- id: chatcmpl-AIc6IK09Tjhfige61IAekQh8FLNLH
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I successfully scraped the URL https://www.fewsats.com/. Here is the scraped data:\n\n### Title\n**Monetize your Files, Databases**\n\n### Content (Markdown)\n```markdown\n![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/66475f66cdbcb00d7bf2fd44_Employee%20data.png)\n\n[Learn More](#)![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/6645a4a7082c15bec7e9fdda_arrow-right-icon.png)\n\n![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/6645a4a7082c15bec7e9fdda_arrow-right-icon.png)\n[Learn More](#)![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/6645a4a7082c15bec7e9fdda_arrow-right-icon.png)\n\n![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/6645a4a7082c15bec7e9fdda_arrow-right-icon.png)\n![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/664773c569f5d9dbac6655a0_Employee%20data-2.png)\n\n![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/664762e683491e5b21c08f31_Employee%20data-3.png)\n[Learn More](#)![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/6645a4a7082c15bec7e9fdda_arrow-right-icon.png)\n\nThe L402 protocol builds on top of HTTP and the Lightning network to enable internet-first, machine friendly paywalls.\n\nInternet-Native Paywalls\n\nThe L402 protocol builds on top of HTTP and the Lightning network to enable internet-first, machine friendly paywalls.\n\nL402 is an open protocol that anyone can build upon, fostering a flourishing ecosystem of innovative solutions.\n\nAI systems are now key consumers of software, driving demand for more intelligent, contextual data and tools. With one integration, Fewsats unlocks a universe of resources to supercharge these AI consumers.\n\n**Revolutionize your bots with Retrieval Augmented Generation (RAG) and in-context information.**\n```\n\n### Metadata\n- **Source URL**: [Fewsats](https://www.fewsats.com/)\n- **Status Code**: 200\n\nIf you need more specific information or further assistance, let me know!', refusal=None, role='assistant', function_call=None, tool_calls=None))]
- created: 1728999726
- model: gpt-4o-mini-2024-07-18
- object: chat.completion
- service_tier: None
- system_fingerprint: fp_e2bde53e6e
- usage: CompletionUsage(completion_tokens=618, prompt_tokens=1210, total_tokens=1828, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0))

</details>

Now that we have the full process on how to add a tool to the agent, let's write a single function that encapsulates the entire process. The agent will be able to call this function to add a new tool.

In [ ]:
#| export

def add_l402_tool(uri: str) -> str:
    """Add a new tool to the agent's toolset."""
    info = get_l402_uri_info(uri)
    r = generate_python_function(info)
    func_code = extract_function_code(contents(r))

    cf = create_func(func_code)
    tools.append(cf)
    return f"tool {cf.__name__} added"

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
sp = """You are a helpful assistant that can add new tools to help users accomplish actions and get information. 
When a user provides an L402 URI, you should add it as a tool right away. If you do not have any tools, say so if the user asks."""
model = "gpt-4o"
tools = [add_l402_tool]
chat = Chat(model, sp=sp, tools=tools)


In [ ]:

print("Received messages:", messages)
msgs = [mk_msg(m['content']) for m in messages[:-1]]
print(msgs)
chat.h = msgs
# Use Cosette's Chat with tool loop, including the conversation history
response = chat.toolloop(pr=messages[-1]['content'], trace_func=pchoice)


In [ ]:
chat.toolloop(pr='scrape wolfync.com/schedule')

It seems that I currently don't have any tools available to perform web scraping tasks. If you have an L402 URI for a scraping tool, you can provide it, and I can add it as a tool to assist with your request.

<details>

- id: chatcmpl-AIcUgLTGZC3X0PygNiOYnOqjR61Af
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="It seems that I currently don't have any tools available to perform web scraping tasks. If you have an L402 URI for a scraping tool, you can provide it, and I can add it as a tool to assist with your request.", refusal=None, role='assistant', function_call=None, tool_calls=None))]
- created: 1729001238
- model: gpt-4o-2024-08-06
- object: chat.completion
- service_tier: None
- system_fingerprint: fp_6b68a8204b
- usage: CompletionUsage(completion_tokens=48, prompt_tokens=118, total_tokens=166, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0))

</details>

In [ ]:
def print_history(h):
    for i,m in enumerate(h):
        print(i,m)

print_history(chat.h)

0 {'role': 'user', 'content': [{'type': 'text', 'text': 'scrape wolfync.com/schedule'}]}
1 ChatCompletionMessage(content="It seems that I currently don't have any tools available to perform web scraping tasks. If you have an L402 URI for a scraping tool, you can provide it, and I can add it as a tool to assist with your request.", refusal=None, role='assistant', function_call=None, tool_calls=None)


In [ ]:
chat.toolloop(pr='Tool to scrape websites: l402://api.fewsats.com/v0/gateway/f12e5deb-b07b-4af4-a4f2-3fbf076228a9/info')

An error occurred: 500 Server Error: Internal Server Error for url: https://api.fewsats.com/v0/gateway/access/f12e5deb-b07b-4af4-a4f2-3fbf076228a9


It seems there was an issue with scraping the website "https://wolfync.com/schedule". The response did not return any data. There could be multiple reasons for this, including access restrictions on the website or issues with the scraping tool. If there's anything else you'd like to try or any other assistance you need, feel free to let me know!

<details>

- id: chatcmpl-AIcVCjQm2sUUoWlPudECWE3KRMOCU
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='It seems there was an issue with scraping the website "https://wolfync.com/schedule". The response did not return any data. There could be multiple reasons for this, including access restrictions on the website or issues with the scraping tool. If there\'s anything else you\'d like to try or any other assistance you need, feel free to let me know!', refusal=None, role='assistant', function_call=None, tool_calls=None))]
- created: 1729001270
- model: gpt-4o-2024-08-06
- object: chat.completion
- service_tier: None
- system_fingerprint: fp_a20a4ee344
- usage: CompletionUsage(completion_tokens=71, prompt_tokens=407, total_tokens=478, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0))

</details>

In [ ]:
[print(i,m) for i,m in enumerate(chat.h)]

0 {'role': 'user', 'content': [{'type': 'text', 'text': 'scrape wolfync.com/schedule'}]}
1 ChatCompletionMessage(content="It seems that I currently don't have any tools available to perform web scraping tasks. If you have an L402 URI for a scraping tool, you can provide it, and I can add it as a tool to assist with your request.", refusal=None, role='assistant', function_call=None, tool_calls=None)
2 {'role': 'user', 'content': [{'type': 'text', 'text': 'Tool to scrape websites: l402://api.fewsats.com/v0/gateway/f12e5deb-b07b-4af4-a4f2-3fbf076228a9/info'}]}
3 ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_f7BxSCx4CFf3akLngiSK16dV', function=Function(arguments='{"uri":"l402://api.fewsats.com/v0/gateway/f12e5deb-b07b-4af4-a4f2-3fbf076228a9/info"}', name='add_l402_tool'), type='function')])
4 {'role': 'tool', 'content': [{'type': 'text', 'text': 'tool scrape_web_url added'}], 'tool_call_id': 'call_

[None, None, None, None, None, None, None, None]

In [ ]:
msgs = chat.h

In [ ]:
chat = Chat(model, sp=sp, tools=tools)
chat.h


[]

In [ ]:
chat.toolloop(pr='scrape https://www.fewsats.com/')

I currently do not have any tools available to perform web scraping. If you can provide an L402 URI, I can add the necessary tool to help accomplish this task.

<details>

- id: chatcmpl-AIcXW0woFpKlT3A3OzdfCDLWbKmJE
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I currently do not have any tools available to perform web scraping. If you can provide an L402 URI, I can add the necessary tool to help accomplish this task.', refusal=None, role='assistant', function_call=None, tool_calls=None))]
- created: 1729001414
- model: gpt-4o-2024-08-06
- object: chat.completion
- service_tier: None
- system_fingerprint: fp_a20a4ee344
- usage: CompletionUsage(completion_tokens=35, prompt_tokens=213, total_tokens=248, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0))

</details>

In [ ]:
chat.h = msgs

In [ ]:
chat.toolloop(pr='scrape https://www.fewsats.com/')

The scraping of "https://www.fewsats.com/" was successful. Here's a brief overview of the content:

- The website discusses the L402 protocol, which builds on top of HTTP and the Lightning network to enable internet-first, machine-friendly paywalls.
- The L402 protocol is an open protocol fostering a flourishing ecosystem of innovative solutions.
- There's a focus on AI systems as key consumers of software, driving demand for more intelligent, contextual data and tools. Fewsats' integration helps unlock resources for these AI consumers.
- It mentions "Retrieval Augmented Generation (RAG)" and the revolutionizing of bots with in-context information.

If you're interested in specific details or have any further questions, let me know!

<details>

- id: chatcmpl-AIcY5xu9yHs5R9w8gmOcZ8eTpq8zT
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The scraping of "https://www.fewsats.com/" was successful. Here\'s a brief overview of the content:\n\n- The website discusses the L402 protocol, which builds on top of HTTP and the Lightning network to enable internet-first, machine-friendly paywalls.\n- The L402 protocol is an open protocol fostering a flourishing ecosystem of innovative solutions.\n- There\'s a focus on AI systems as key consumers of software, driving demand for more intelligent, contextual data and tools. Fewsats\' integration helps unlock resources for these AI consumers.\n- It mentions "Retrieval Augmented Generation (RAG)" and the revolutionizing of bots with in-context information.\n\nIf you\'re interested in specific details or have any further questions, let me know!', refusal=None, role='assistant', function_call=None, tool_calls=None))]
- created: 1729001449
- model: gpt-4o-2024-08-06
- object: chat.completion
- service_tier: None
- system_fingerprint: fp_a20a4ee344
- usage: CompletionUsage(completion_tokens=147, prompt_tokens=1379, total_tokens=1526, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0))

</details>

In [ ]:
msgs[1]

ChatCompletionMessage(content="It seems that I currently don't have any tools available to perform web scraping tasks. If you have an L402 URI for a scraping tool, you can provide it, and I can add it as a tool to assist with your request.", refusal=None, role='assistant', function_call=None, tool_calls=None)